IMPORT DEPENDENCIES

In [2]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

LOAD ENV

In [117]:
env_name= "CartPole-v0"
env= gym.make(env_name, render_mode="human")

In [119]:
episodes = 5 #number of times the whole environment is run
for episode in range(1, episodes + 1):
    state= env.reset() # reset the environment to the initial state.... it will return the initial state value which is the first observation
    done = False # done is a boolean that indicates whether the episode has ended
    score = 0 # initialize the score for the episode
    
    while not done:
        env.render() # render   the environment to visualize the actions taken
        action= env.action_space.sample() # take a random action from the action space
        state, reward, terminated , done, info  = env.step(action)  # apply the action to the environment
        # state is the new state after taking the action
        # reward is the reward received for taking the action
        # terminated is a boolean that indicates whether the episode has ended due to a terminal state
        # done is a boolean that indicates whether the episode has ended
        # info is a dictionary that contains additional information about the environment
        score += reward 

    print(f"Episode {episode} Score: {score}")
env.close() # close the render frame 

Episode 1 Score: 15.0
Episode 2 Score: 26.0
Episode 3 Score: 23.0
Episode 4 Score: 12.0
Episode 5 Score: 43.0


In [43]:
env.action_space.sample()

1

In [109]:
env.step(1)

(array([ 1.0918093,  4.37141  , -1.7520808, -8.110683 ], dtype=float32),
 0.0,
 True,
 False,
 {})

UNDERSTANDING ENV

In [136]:
env.action_space.sample()

0

In [120]:
env.action_space

Discrete(2)

In [137]:
env.observation_space

Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)

In [138]:
env.observation_space.sample()

array([-0.45313802, -0.63500464, -0.2454992 , -0.98092854], dtype=float32)

TRAINING THE MODEL

In [157]:
log_path = os.path.join('Training', 'Logs')

In [158]:
log_path

'Training\\Logs'

In [ ]:
env = gym.make(env_name, render_mode="human") #render_mode="human" is used to visualize the environment
env = DummyVecEnv([lambda: env])  # Wrap the environment in a DummyVec because Stable Baselines3 requires a vectorized environment for training
model = PPO("MlpPolicy", env, verbose=1, tensorboard_log=log_path)  # Create the PPO model with MLP policy which is a Multi-Layer Perceptron
# verbose=1 will print the training progress
# tensorboard_log=log_path will log the training progress to the specified path for visualization in TensorBoard

In [160]:
model.learn(total_timesteps=20000)  # Train the model for 20,000 timesteps

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 45   |
|    iterations      | 1    |
|    time_elapsed    | 45   |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 45          |
|    iterations           | 2           |
|    time_elapsed         | 90          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008916764 |
|    clip_fraction        | 0.0875      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00335    |
|    learning_rate        | 0.0003      |
|    loss                 | 4.5         |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0125     |
|    value_loss           | 50.8        |
-----------------------------------------
---

In [ ]:
env.close()  # close the environment

SAVING THE MODEL

In [162]:
PPO_path = os.path.join('Training', 'SavedModels', 'PPO_CartPole_Model')

In [163]:
model.save(PPO_path)  # Save the model to the specified path

In [165]:
del model  # delete the model to free up memory

In [176]:
model = PPO.load(PPO_path, env=env)  # Load the model from the specified path

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


EVALUATION

In [171]:
evaluate_policy( model, env, n_eval_episodes= 10, render=True)  # Evaluate the model on 10 episodes and render the environment)
# output will be the mean reward and standard deviation of the reward over the 10 episodes

(200.0, 0.0)

In [172]:
env.close()  # close the environment

TESTING

In [179]:
episodes = 5 
env= gym.make(env_name, render_mode="human")  # create the environment with human rendering
env = DummyVecEnv([lambda: env])  

for episode in range(1, episodes + 1): 
    state = env.reset()  # reset the environment to the initial state
    done = False  # done is a boolean that indicates whether the episode has ended
    score = 0  # initialize the score for the episode
    
    while not done:
        env.render()
        action, _states = model.predict(state)  # use the model to predict the action to take based on the current state
        state, reward, done, info = env.step(action)  # apply the action to the environment
        score += reward  # accumulate the score

    print(f"Episode {episode} Score: {score}")  # print the score for the episode

Episode 1 Score: [200.]
Episode 2 Score: [200.]
Episode 3 Score: [127.]
Episode 4 Score: [200.]
Episode 5 Score: [200.]


In [180]:
env.close()

In [187]:
env = gym.make(env_name, render_mode="human")  # create the environment with human rendering
env = DummyVecEnv([lambda: env])  # Wrap the environment in a DummyVec
obs=env.reset()  

In [193]:
model.predict(obs)  

(array([0], dtype=int64), None)

VIEWING LOGS IN TENSORBOARD

In [199]:
training_log_path = os.path.join(log_path, 'PPO_1')
training_log_path

'Training\\Logs\\PPO_1'

In [200]:
!tensorboard --logdir={training_log_path}  

^C
